# Hartree-Fock VQE Demo

In [ ]:
import tqdm
import pickle

import numpy as np
import scipy as sp
import numpy_ml

from molecular_data import MolecularData, obi_basis_rotation, tbi_basis_rotation, generate_hamiltonian
from QuICT.algorithm.quantum_machine_learning.model.VQE.operators.encoder import JordanWigner
from QuICT.algorithm.quantum_machine_learning.model.VQE import HartreeFockVQENet
from QuICT.algorithm.quantum_machine_learning.utils import Hamiltonian

## Load the data

In [ ]:
moldir = "./molecular_data"
molfile = moldir + "/H6_sto-3g_singlet_linear_r-1.3.hdf5"
moldata = MolecularData(molfile)

overlap = np.load(moldir + "/overlap.npy")
Hcore = np.load(moldir + "/h_core.npy")
two_electron_integral = np.einsum("psqr", np.load(moldir + "/tei.npy"))  # (1, 1, 0, 0)

_, X = sp.linalg.eigh(Hcore, overlap)
obi = obi_basis_rotation(Hcore, X)
tbi = tbi_basis_rotation(two_electron_integral, X)
molecular_hamiltonian = generate_hamiltonian(moldata.nuclear_repulsion, obi, tbi)

## Convert the Hamiltonian

In [ ]:
fermi_op = molecular_hamiltonian.get_fermion_operator()
orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
qubit_op = JordanWigner(orbitals).encode(fermi_op)
hamiltonian = Hamiltonian(qubit_op.to_hamiltonian())

ham_str = pickle.dumps(hamiltonian)
with open('hamiltonian', 'wb') as f:
    f.write(ham_str)

## Calculate the ground energy

In [ ]:
MAX_ITERS = 1000
LR = 0.1

orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
with open('hamiltonian', 'rb') as f:
    hamiltonian = pickle.load(f)
optim = numpy_ml.neural_nets.optimizers.Adam(lr=LR)
hfvqe_net = HartreeFockVQENet(orbitals, electrons, optim, hamiltonian)
energy = []

loader = tqdm.trange(MAX_ITERS, desc="Training", leave=True)
for it in loader:
    state, loss = hfvqe_net.run_step()
    print(hfvqe_net.params)
    for g in hfvqe_net._circuit.gates:
        print(g.type, g.parg)
    loader.set_postfix(loss=loss.item())

print(energy)
print(hfvqe_net.params)
print(float(loss))

Training:   2%|▏         | 15/1000 [00:35<40:06,  2.44s/it, loss=-2.37]

Variable(pargs=[ 1.14236939 -0.5363157   0.48602756 -0.25269843 -1.04860357 -0.207161
  0.73347106 -1.35018747  0.32289595 -0.67602757 -0.26316251  0.0212294
  0.05507979  0.49805427  0.79949031  0.74133209 -0.52116604 -0.83382458
  0.72988334  0.0348347   0.47782987  0.56354866 -0.29876385  1.3565192
  0.81760093 -0.66178981  2.21837697 -1.38876101  2.76614682 -0.45465856
  0.13082007  0.83582548 -0.77481758 -0.11606491  0.30417633 -1.96288888], grads=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], identity=612ce6a025f811ee8f96f7b57e969e00, shape=(36,))
GateType.x None
GateType.x None
GateType.x None
GateType.x None
GateType.x None
GateType.x None
GateType.sqiswap None
GateType.rz Variable(pargs=-1.1423693853068075, grads=-1.0, identity=612ce6a025f811ee8f96f7b57e969e000, , shape=())
GateType.rz Variable(pargs=4.2839620388966, grads=0.0, identity=612ce6a025f811ee8f96f7b57e969e000, , shape=())
GateType.sqiswap None
GateType